In [123]:
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from stable_baselines.common.policies import MlpPolicy as PPO_MLP
from stable_baselines.deepq.policies import MlpPolicy as DQN_MLP
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2
from stable_baselines import DQN
from environment.Stock_trading_env import Stock_trading
import altair as alt
# from experiment.visualization import visualize

In [124]:
import warnings
warnings.filterwarnings("ignore")

In [125]:
stock_list = ['sh.000001','sz.000002','sz.002371','sh.600004','sh.600030','sh.600036','sh.600276','sh.600519','sh.600900','sh.601318']

In [126]:
def find_file(path, name):
    for root, dirs, files in os.walk(path):
        for fname in files:
            if name in fname:
                return os.path.join(root, fname)

In [163]:
def stock_trade(stock_file):
    daily_wealth = []
    history = []
    df = pd.read_csv(stock_file)
    df = df.sort_values('date')

    env = DummyVecEnv([lambda: Stock_trading(df)])

    model = PPO2(PPO_MLP, env, verbose=0, tensorboard_log='.\\log')
    model.learn(total_timesteps=int(1e4))

    df_test = pd.read_csv(stock_file.replace('train', 'test'))

    env = DummyVecEnv([lambda: Stock_trading(df_test)])
    obs = env.reset()
    for i in range(len(df_test) - 1):
        action, _states = model.predict(obs)
#         print('ACTION:',action)
        obs, rewards, done, info = env.step(action)
        wealth, action_name = env.render()
        daily_wealth.append(wealth)
        history.append(action_name)
        if done:
            break
    return daily_wealth,history

In [164]:
def single_stock(stock_code):
    stock_file = find_file('.\\data\\train', str(stock_code))
    df_test = pd.read_csv(stock_file.replace('train', 'test'))

    daily_wealth,history = stock_trade(stock_file)
    vis_df=df_test[['date','close']]
    vis_df['close']=vis_df['close']/vis_df.loc[0,'close']
    vis_df['date'] = pd.to_datetime(vis_df['date'],infer_datetime_format=True)
    vis_df['wealth']=pd.DataFrame(([daily_wealth[0]]+daily_wealth)/daily_wealth[0])
    vis_df['action']=pd.DataFrame(['HOLD']+history)
    return vis_df

In [165]:
# def multi_stock():
#     group_result = []
#     stock_list = ['sh.000001','sz.000002','sz.002371','sh.600004','sh.600030','sh.600036','sh.600276','sh.600519','sh.600900','sh.601318']

#     for code in stock_list:
#         stock_file = find_file('.\\data\\train', str(code))
#         if stock_file:
#             try:
#                 profits = single_stock(stock_file)
#                 group_result.append(profits)
#             except Exception as e:
#                 print(e)

#     with open('multi_stock.pkl', 'wb') as f:
#         pickle.dump(group_result, f)

In [166]:
vis_df = single_stock('sh.20')

----------------------------------------------------------------------------------------------------
Step: 1
Action:BUY 
Balance:397405.496100208
Share(s): 21567 
Avg cost for held shares: 27.940580697352065 
Net wealth: 1009476.956100208 (Max net wealth: 1009476.956100208)
Profit: 9476.956100207986
----------------------------------------------------------------------------------------------------
Step: 2
Action:HOLD 
Balance:397405.496100208
Share(s): 21567 
Avg cost for held shares: 27.940580697352065 
Net wealth: 1001497.1661002081 (Max net wealth: 1009476.956100208)
Profit: 1497.1661002080655
----------------------------------------------------------------------------------------------------
Step: 3
Action:BUY 
Balance:166480.53081062256
Share(s): 29881 
Avg cost for held shares: 27.894631009316203 
Net wealth: 1002550.9108106226 (Max net wealth: 1009476.956100208)
Profit: 2550.9108106226195
------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
Step: 109
Action:HOLD 
Balance:4.031082484056242
Share(s): 32612 
Avg cost for held shares: 25.044809222245327 
Net wealth: 824761.511082484 (Max net wealth: 1057699.3354820374)
Profit: -175238.48891751596
----------------------------------------------------------------------------------------------------
Step: 110
Action:HOLD 
Balance:4.031082484056242
Share(s): 32612 
Avg cost for held shares: 25.044809222245327 
Net wealth: 832262.271082484 (Max net wealth: 1057699.3354820374)
Profit: -167737.72891751595
----------------------------------------------------------------------------------------------------
Step: 111
Action:HOLD 
Balance:4.031082484056242
Share(s): 32612 
Avg cost for held shares: 25.044809222245327 
Net wealth: 825087.631082484 (Max net wealth: 1057699.3354820374)
Profit: -174912.36891751597
-------------------------------------------------------------------------------

Avg cost for held shares: 30.494659795799674 
Net wealth: 1511603.904783239 (Max net wealth: 1511603.904783239)
Profit: 511603.9047832389
----------------------------------------------------------------------------------------------------
Step: 210
Action:HOLD 
Balance:12.144783238822129
Share(s): 32676 
Avg cost for held shares: 30.494659795799674 
Net wealth: 1472065.9447832387 (Max net wealth: 1511603.904783239)
Profit: 472065.9447832387
----------------------------------------------------------------------------------------------------
Step: 211
Action:HOLD 
Balance:12.144783238822129
Share(s): 32676 
Avg cost for held shares: 30.494659795799674 
Net wealth: 1473699.744783239 (Max net wealth: 1511603.904783239)
Profit: 473699.744783239
----------------------------------------------------------------------------------------------------
Step: 212
Action:HOLD 
Balance:12.144783238822129
Share(s): 32676 
Avg cost for held shares: 30.494659795799674 
Net wealth: 1421418.1447832389 (Max 

----------------------------------------------------------------------------------------------------
Step: 272
Action:BUY 
Balance:26.356912102302886
Share(s): 34441 
Avg cost for held shares: 34.16149447350634 
Net wealth: 933033.0469121023 (Max net wealth: 1511603.904783239)
Profit: -66966.95308789774
----------------------------------------------------------------------------------------------------
Step: 273
Action:SELL 
Balance:747141.1392414895
Share(s): 8259 
Avg cost for held shares: 34.16149447350634 
Net wealth: 981283.7892414895 (Max net wealth: 1511603.904783239)
Profit: -18716.21075851051
----------------------------------------------------------------------------------------------------
Step: 274
Action:BUY 
Balance:150312.5983888678
Share(s): 28907 
Avg cost for held shares: 30.406763887961755 
Net wealth: 973873.0283888677 (Max net wealth: 1511603.904783239)
Profit: -26126.971611132263
-------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
Step: 336
Action:HOLD 
Balance:20.31308157404419
Share(s): 33675 
Avg cost for held shares: 37.375928605082315 
Net wealth: 1693872.8130815742 (Max net wealth: 1776039.8130815742)
Profit: 693872.8130815742
----------------------------------------------------------------------------------------------------
Step: 337
Action:HOLD 
Balance:20.31308157404419
Share(s): 33675 
Avg cost for held shares: 37.375928605082315 
Net wealth: 1660197.8130815742 (Max net wealth: 1776039.8130815742)
Profit: 660197.8130815742
----------------------------------------------------------------------------------------------------
Step: 338
Action:HOLD 
Balance:20.31308157404419
Share(s): 33675 
Avg cost for held shares: 37.375928605082315 
Net wealth: 1575336.8130815742 (Max net wealth: 1776039.8130815742)
Profit: 575336.8130815742
-------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
Step: 421
Action:HOLD 
Balance:38.212006031302735
Share(s): 39132 
Avg cost for held shares: 52.187025467912676 
Net wealth: 1807545.2920060311 (Max net wealth: 2168733.652006031)
Profit: 807545.2920060311
----------------------------------------------------------------------------------------------------
Step: 422
Action:HOLD 
Balance:38.212006031302735
Share(s): 39132 
Avg cost for held shares: 52.187025467912676 
Net wealth: 1813415.0920060314 (Max net wealth: 2168733.652006031)
Profit: 813415.0920060314
----------------------------------------------------------------------------------------------------
Step: 423
Action:HOLD 
Balance:38.212006031302735
Share(s): 39132 
Avg cost for held shares: 52.187025467912676 
Net wealth: 1861156.1320060315 (Max net wealth: 2168733.652006031)
Profit: 861156.1320060315
-------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
Step: 490
Action:HOLD 
Balance:5.286695067479741
Share(s): 40539 
Avg cost for held shares: 42.7553489610382 
Net wealth: 1706291.7966950678 (Max net wealth: 2168733.652006031)
Profit: 706291.7966950678
----------------------------------------------------------------------------------------------------
Step: 491
Action:HOLD 
Balance:5.286695067479741
Share(s): 40539 
Avg cost for held shares: 42.7553489610382 
Net wealth: 1705075.6266950676 (Max net wealth: 2168733.652006031)
Profit: 705075.6266950676
----------------------------------------------------------------------------------------------------
Step: 492
Action:HOLD 
Balance:5.286695067479741
Share(s): 40539 
Avg cost for held shares: 42.7553489610382 
Net wealth: 1717642.7166950675 (Max net wealth: 2168733.652006031)
Profit: 717642.7166950675
----------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
Step: 554
Action:HOLD 
Balance:28.054924028692767
Share(s): 32034 
Avg cost for held shares: 67.79131140155472 
Net wealth: 2038671.8149240287 (Max net wealth: 2547367.2111051194)
Profit: 1038671.8149240287
----------------------------------------------------------------------------------------------------
Step: 555
Action:HOLD 
Balance:28.054924028692767
Share(s): 32034 
Avg cost for held shares: 67.79131140155472 
Net wealth: 2143422.9949240284 (Max net wealth: 2547367.2111051194)
Profit: 1143422.9949240284
----------------------------------------------------------------------------------------------------
Step: 556
Action:SELL 
Balance:848866.2763736899
Share(s): 19206 
Avg cost for held shares: 67.79131140155472 
Net wealth: 2112621.07637369 (Max net wealth: 2547367.2111051194)
Profit: 1112621.0763736898
-------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
Step: 639
Action:HOLD 
Balance:87389.46314551056
Share(s): 31924 
Avg cost for held shares: 63.50379322910698 
Net wealth: 1982717.3431455104 (Max net wealth: 2547367.2111051194)
Profit: 982717.3431455104
----------------------------------------------------------------------------------------------------
Step: 640
Action:BUY 
Balance:53148.92914780511
Share(s): 32499 
Avg cost for held shares: 63.43381731880109 
Net wealth: 2049887.489147805 (Max net wealth: 2547367.2111051194)
Profit: 1049887.489147805
----------------------------------------------------------------------------------------------------
Step: 641
Action:HOLD 
Balance:53148.92914780511
Share(s): 32499 
Avg cost for held shares: 63.43381731880109 
Net wealth: 1998214.0791478052 (Max net wealth: 2547367.2111051194)
Profit: 998214.0791478052
------------------------------------------------------------------------------------

Step: 709
Action:SELL 
Balance:1811719.5568450121
Share(s): 11760 
Avg cost for held shares: 67.13792922234 
Net wealth: 2752519.556845012 (Max net wealth: 2765462.405615761)
Profit: 1752519.5568450121
----------------------------------------------------------------------------------------------------
Step: 710
Action:BUY 
Balance:1622430.232994369
Share(s): 14124 
Avg cost for held shares: 69.30270259879366 
Net wealth: 2752208.992994369 (Max net wealth: 2765462.405615761)
Profit: 1752208.992994369
----------------------------------------------------------------------------------------------------
Step: 711
Action:BUY 
Balance:1138379.834529309
Share(s): 19733 
Avg cost for held shares: 74.13377438658195 
Net wealth: 2841929.724529309 (Max net wealth: 2841929.724529309)
Profit: 1841929.7245293092
----------------------------------------------------------------------------------------------------
Step: 712
Action:BUY 
Balance:33.276552053168416
Share(s): 32236 
Avg cost for held shares

In [167]:
vis1=vis_df[['date','close','action']]
vis1['category']='close'
vis1.rename(columns={'close':'Net Worth'},inplace=True)

In [168]:
vis2=vis_df[['date','wealth']]
vis2['action']='HOLD'
vis2['category']='wealth'
vis2.rename(columns={'wealth':'Net Worth'},inplace=True)

In [169]:
new_vis=pd.concat((vis1,vis2),axis=0)

In [170]:
def visualize(df):
    line_df=df[['date','Net Worth','category']]
#     scale = alt.Scale(domain=(min(df['Net Worth']) - 0.2, max(df['Net Worth']) + 0.2),clamp=True)
    
    nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['date'], empty='none')
    
    line = alt.Chart(line_df).mark_line(interpolate='basis').encode(x='date:T',y='Net Worth:Q',color='category:N')
    
    action_points = alt.Chart(df).transform_filter(alt.datum.action != 'HOLD').mark_point(filled=True).encode(x=alt.X('date:T', axis=alt.Axis(title='Date')),y=alt.Y('Net Worth', axis=alt.Axis(format='.4f', title='Net Worth')),color='action').interactive(bind_y=False)
    
    selectors = alt.Chart(line_df).mark_point().encode(x='date:T',opacity=alt.value(0),).add_selection(nearest)
    
    # Draw points on the line, and highlight based on selection
    points = line.mark_point().encode(opacity=alt.condition(nearest, alt.value(1), alt.value(0)))
    
    # Draw text labels near the points, and highlight based on selection
    text = line.mark_text(align='left', dx=5, dy=-5).encode(text=alt.condition(nearest, 'Net Worth:Q', alt.value(' ')))
    
    # Draw a rule at the location of the selection
    rules = alt.Chart(line_df).mark_rule(color='gray').encode(x='date:T',).transform_filter(nearest)
    
    chart = alt.layer(line,selectors, points,rules,text,action_points,title="Portfolio Actions").properties(height=300, width=800)
    
    return chart

In [171]:
chart=visualize(new_vis)
chart

alt.LayerChart(...)